In [1]:
import os
import json
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import string
import subprocess

In [2]:
train_dir = "./Data/Training/"
test_dir = "./Data/Performance Test/"
eval_dir = "./Data/Validation/"

### Helper functions

In [3]:
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]
        if (len(files) > 0):                                                                                          
            for file in files:
                if file.endswith('.json'):
                    r.append(os.path.join(subdir, file))                   
    return r

def list_single_dir_files(dir):
    for (dirpath, dirnames, filenames) in os.walk(dir):
        return filenames
        break
        
        
def leaf_dirs(dir):
    max_depth = 0
    leaf_dir = []
    for dirpath, dirnames, filenames in os.walk(dir):
        depth= len(dirpath.split(os.sep))
        if max_depth < depth:
            max_depth= depth
            leaf_dir = [dirpath]
        elif max_depth == depth:
            leaf_dir.append(dirpath)
    return(leaf_dir)


def decode_dump(dir):
    files=list_single_dir_files(dir)
    for file in files:
        command=[]
        if file.endswith('.raw'):
            command=["xxd","-c8","-a",dir+'/'+file,dir+"/"+file[:-3]+"txt"]
            command[3]=command[3].replace('\\', '/')
            command[4]=command[4].replace('\\', '/')
            subprocess.call(command)
            

def single_dir_offset(dir):
    json_files=[]
    for pos_json in os.listdir(dir):
        if pos_json.endswith('.json'):
            json_files.append(pos_json)  
    dir_res=[]        
    for json_file in json_files:        
        data=None
        result=[]
        f=open(dir+"/"+str(json_file))
        data=json.load(f)
        heap_start=data['HEAP_START']

        keys=[data['KEY_A_ADDR'],data['KEY_B_ADDR'],data['KEY_C_ADDR'],data['KEY_D_ADDR'],data['KEY_E_ADDR'],data['KEY_F_ADDR']]
        lengths=[data['KEY_A_LEN'],data['KEY_B_LEN'],data['KEY_C_LEN'],data['KEY_D_LEN'],data['KEY_E_LEN'],data['KEY_F_LEN']]
        result=[hex(int(e, 16)-int(heap_start, 16)) for e in keys]
        for i in range(len(lengths)):
            if int(lengths[i])==0:
                result[i]= None
        dir_res.append(result)
    return dir_res
               

def plot_subs(df,dir):
    fig, ((ax0, ax1, ax2), (ax3, ax4, ax5)) = plt.subplots(nrows=2, ncols=3,  figsize = (16,6))
    fig.suptitle('Distributions of Key address offset in '+str(dir[74:]))
    if df['A'].unique()[0] :
        ax0.hist(df['A'],  histtype='bar')
    ax0.set_title('A')

    if df['B'].unique()[0] :
        ax1.hist(df['B'], histtype='bar')
    ax1.set_title('B')
    
    if df['C'].unique()[0] :
        ax2.hist(df['C'],histtype='bar')
    ax2.set_title('C')

    if df['D'].unique()[0] :
        ax3.hist(df['D'], histtype='bar')
    ax3.set_title('D')

    if df['F'].unique()[0] :
        ax4.hist(df['E'], histtype='bar')
    ax4.set_title('E')

    if df['F'].unique()[0] :
        ax5.hist(df['F'],  histtype='bar')
    ax5.set_title('F')

    fig.tight_layout()
    plt.show()


### List of all json files

In [4]:
all_files=list_files(mydir)

NameError: name 'mydir' is not defined

### List of all leaf directories

In [4]:

train_leaf_dir=leaf_dirs(train_dir)
test_leaf_dir=leaf_dirs(test_dir)
eval_leaf_dir=leaf_dirs(eval_dir)

In [ ]:
# custom_dir='C:/Users/omars/OneDrive/Bureau/Thesis/Training/basic\\V_8_8_P1\\64'
# decode_dump(custom_dir)

In [5]:
leaf_dir=[]
leaf_dir.extend(train_leaf_dir)
leaf_dir.extend(test_leaf_dir)
leaf_dir.extend(eval_leaf_dir)

### Decoding hex dumps

In [6]:
leaf_dir

['./Data/Training/scp/V_8_0_P1/64',
 './Data/Training/scp/V_8_0_P1/24',
 './Data/Training/scp/V_8_0_P1/32',
 './Data/Training/scp/V_8_0_P1/16',
 './Data/Training/scp/V_7_8_P1/64',
 './Data/Training/scp/V_7_8_P1/24',
 './Data/Training/scp/V_7_8_P1/32',
 './Data/Training/scp/V_7_8_P1/16',
 './Data/Training/port-forwarding/V_8_0_P1/64',
 './Data/Training/port-forwarding/V_8_0_P1/24',
 './Data/Training/port-forwarding/V_8_0_P1/32',
 './Data/Training/port-forwarding/V_8_0_P1/16',
 './Data/Training/port-forwarding/V_7_8_P1/64',
 './Data/Training/port-forwarding/V_7_8_P1/24',
 './Data/Training/port-forwarding/V_7_8_P1/32',
 './Data/Training/port-forwarding/V_7_8_P1/16',
 './Data/Training/basic/V_6_7_P1/24',
 './Data/Training/basic/V_6_7_P1/32',
 './Data/Training/basic/V_6_7_P1/16',
 './Data/Training/basic/V_6_8_P1/64',
 './Data/Training/basic/V_6_8_P1/24',
 './Data/Training/basic/V_6_8_P1/32',
 './Data/Training/basic/V_6_8_P1/16',
 './Data/Training/basic/V_6_2_P1/24',
 './Data/Training/basic/

In [7]:
%%time
for directory in leaf_dir:
    decode_dump(directory)

CPU times: user 19.5 s, sys: 2min 13s, total: 2min 32s
Wall time: 36min 29s


### Keys addresses offset over all files

In [ ]:
for json_file in all_files:
    result=[]
    data=None
    f=open(json_file)
    data=json.load(f)
    heap_start=data['HEAP_START']
    keys=[None,None,None,None,None,None]
    
    try:
        if int(data['KEY_A_LEN']) > 0:
            keys[0]=data['KEY_A_ADDR']
        else:
            pass      
    except :
        pass
    
    try:
        if int(data['KEY_B_LEN']) > 0:
            keys[1]=data['KEY_B_ADDR']
        else:
            pass      
    except :
        pass
    
    try:
        if int(data['KEY_C_LEN']) > 0:
            keys[2]=data['KEY_C_ADDR']
        else:
            pass      
    except :
        pass
    
    try:
        if int(data['KEY_D_LEN']) > 0:
            keys[3]=data['KEY_D_ADDR']
        else:
            pass      
    except :
        pass
    
    try:
        if int(data['KEY_E_LEN']) > 0:
            keys[4]=data['KEY_E_ADDR']
        else:
            pass      
    except :
        pass


    try:
        if int(data['KEY_F_LEN']) > 0:
            keys[5]=data['KEY_F_ADDR']
        else:
            pass      
    except :
        pass


    for e in keys:
        if e:
            result.append(hex(int(e, 16)-int(heap_start, 16)))
        else:
            result.append(None)
    print(str(json_file)[74:],'  ',result,'\n')



### ssh key address offsets ditributions

In [ ]:
for k in range(len(leaf_dir_reduced)):
    print("--------------------------------------- Version : "+str(leaf_dir[k][-9:-3])+" / Encryption key length : "+str(leaf_dir[k][-2:])+" ---------------------------------------")
    values=single_dir_offset(leaf_dir[k])
    m_values=np.array([np.array(value) for value in values])
    df = pd.DataFrame(m_values,columns=['A','B','C','D','E','F'])
    plot_subs(df,leaf_dir[k])

        